In [111]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Моделирование событий пользователей.

## 1. Регистрация пользователя (register)
- Событие регистрации создается для каждого пользователя с разной датой в пределах 30 дней от начальной даты.
- Это событие не привязано к конкретному продукту (`product = None`).

## 2. Визиты на страницы продуктов (pageVisit)
- После регистрации пользователь посещает страницу одного или нескольких продуктов.
- Выбор продуктов осуществляется случайным образом из списка `product_ids`.
- Событие `pageVisit` учитывает разные продукты, что имитирует просмотр различных приложений в сторе.

## 3. Загрузка приложения (download)
- 80% пользователей совершают загрузку (регулируется вероятностью 0.8).
- Событие `download` связано с конкретным продуктом, выбранным случайным образом.
- Пользователь может загрузить продукт в тот же день или в течение двух недель после регистрации, имитируя быструю или отложенную загрузку.

## 4. Покупка приложения (buy)
- 50% пользователей покупают приложение после его загрузки (регулируется вероятностью 0.5).
- У покупки есть случайная задержка от 1 до 5 дней после загрузки.
- Покупка также привязана к конкретному продукту, имитируя возможные покупки разных приложений.

## 5. Обновление приложения (update)
- 30% пользователей совершают обновление приложения (регулируется вероятностью 0.3).
- Обновление происходит в течение 1–7 дней после покупки.
- Каждое обновление связано с продуктом, что имитирует выбор для обновления конкретного приложения.

## 6. Первичный запуск приложения (launch)
- Для каждого пользователя создается событие запуска `launch`, которое происходит в пределах 3 дней после регистрации.
- `launch` не привязан к продукту (`product = None`), так как он предствляет запуск стор приложения.
ия.
ск стор приложения.


In [127]:
# Задаем параметры
np.random.seed(42)
user_count = 10  # количество уникальных пользователей
product_ids = ['prod_1', 'prod_2', 'prod_3']  # несколько идентификаторов продуктов
start_date = datetime(2023, 3, 1)

# Функция для генерации случайного события
def generate_event(user_id, product, event_name, event_time):
    return {
        'userId': user_id,
        'eventName': event_name,
        'time': event_time,
        'product': product
    }

# Список для хранения событий
events = []

# Генерация событий для каждого пользователя
for user_id in range(1, user_count + 1):
    # Дата регистрации пользователя
    register_time = start_date + timedelta(days=np.random.randint(0, 30))
    events.append(generate_event(user_id, None, 'register', register_time))
    
    # Добавляем визиты на страницы продуктов
    for product in product_ids:
        page_visit_time = register_time + timedelta(days=np.random.randint(1, 7))
        events.append(generate_event(user_id, product, 'pageVisit', page_visit_time))
    
    # Генерация событий для загрузки, покупки, обновления
    if np.random.rand() < 0.8:  # 80% шанс, что пользователь скачает приложение
        download_time = register_time + timedelta(days=np.random.randint(1, 14))
        events.append(generate_event(user_id, np.random.choice(product_ids), 'download', download_time))

    if np.random.rand() < 0.5:  # 50% шанс, что пользователь купит приложение
        buy_time = download_time + timedelta(days=np.random.randint(1, 5))
        events.append(generate_event(user_id, np.random.choice(product_ids), 'buy', buy_time))
        
    if np.random.rand() < 0.3:  # 30% шанс, что пользователь обновит приложение
        update_time = buy_time + timedelta(days=np.random.randint(1, 7))
        events.append(generate_event(user_id, np.random.choice(product_ids), 'update', update_time))
    
    # Первичный запуск приложения
    launch_time = register_time + timedelta(days=np.random.randint(0, 3))
    events.append(generate_event(user_id, None, 'launch', launch_time))

# Создаем DataFrame
df = pd.DataFrame(events)

# Сортируем по времени для удобства
df.sort_values(by='time', inplace=True)

# Просмотр сгенерированного датасета
print(df.head(20))

    userId  eventName       time product
31       6   register 2023-03-02    None
37       6     launch 2023-03-03    None
7        2   register 2023-03-04    None
12       2     launch 2023-03-04    None
32       6  pageVisit 2023-03-06  prod_1
34       6  pageVisit 2023-03-06  prod_3
33       6  pageVisit 2023-03-06  prod_2
30       5     launch 2023-03-07    None
26       5   register 2023-03-07    None
0        1   register 2023-03-07    None
8        2  pageVisit 2023-03-07  prod_1
6        1     launch 2023-03-07    None
65      10     launch 2023-03-08    None
60      10   register 2023-03-08    None
29       5  pageVisit 2023-03-08  prod_3
63      10  pageVisit 2023-03-09  prod_3
27       5  pageVisit 2023-03-09  prod_1
10       2  pageVisit 2023-03-09  prod_3
9        2  pageVisit 2023-03-10  prod_2
3        1  pageVisit 2023-03-10  prod_3


In [130]:
df.to_csv('../data/data.csv', index=False)

# Приступаем к когортному анализу

In [450]:
user_actions = pd.read_csv('../data/data.csv', parse_dates=['time'])

In [451]:
register_df = user_actions[user_actions['eventName'] == 'register'][['userId', 'time']].sort_values('time').rename(columns={'time': 'week'})
download_df = user_actions[user_actions['eventName'] == 'download'][['userId', 'time']].sort_values('time').drop_duplicates(subset='userId', keep='first').rename(columns={'time': 'week'})

In [452]:
register_df.week = register_df.week.apply(lambda x: x - pd.to_timedelta(x.weekday(), unit='d')).dt.date
download_df.week = download_df.week.apply(lambda x: x - pd.to_timedelta(x.weekday(), unit='d')).dt.date

In [453]:
amount_of_reg_df = pd.DataFrame(register_df.groupby('week', as_index=False).userId.count().rename(columns={'userId': 'registrations'}))

In [454]:
down_reg_df = register_df.merge(download_df, on=['userId', 'week']) \
    .groupby('week', as_index=False).userId.count() \
    .rename(columns={'userId': 'first_downloads'}) \
    .merge(amount_of_reg_df)

In [455]:
down_reg_df['CR'] = (down_reg_df.first_downloads / down_reg_df.registrations).mask(down_reg_df['registrations'] == 0, 0)

In [456]:
conversion_rate_df = down_reg_df[['week', 'registrations', 'CR']].rename(columns={'registrations': 'users'})

In [457]:
cr_df = amount_of_reg_df.rename(columns={'registrations': 'users'}).merge(conversion_rate_df, how='left').fillna(0)

In [458]:
cr_df

,week,users,CR
0,2023-02-27,2,0.00
1,2023-03-06,4,0.25
2,2023-03-27,4,0.00
